In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [198]:
def get_area(mask):
    # Morphological closing to get whole particles; opening to get rid of noise
    img_mop = cv.morphologyEx(mask, cv.MORPH_CLOSE, cv.getStructuringElement(cv.MORPH_ELLIPSE, (7, 7)))
    img_mop = cv.morphologyEx(img_mop, cv.MORPH_OPEN, cv.getStructuringElement(cv.MORPH_ELLIPSE, (15, 15)))

    # Find contours
    cnts, _ = cv.findContours(img_mop, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)

    # Get bounding rectangles for the scale and the particles
    if len(cnts) > 0:  
        areas = []
        for cnt in cnts:
            areas.append(cv.contourArea(cnt))
        return max(areas)
    else: # No blobs were detected.
        return 0
    
def return_orientation(IMG):
    # blur images. If slow performance, can probably get away with remove some blurs
    blur = cv.blur(IMG,(5,5))
    blur0 = cv.medianBlur(blur,5)
    blur1= cv.GaussianBlur(blur0,(5,5),0)
    blur2= cv.bilateralFilter(blur1,9,75,75)
    
    # convert to HSV
    hsv = cv.cvtColor(blur1, cv.COLOR_BGR2HSV)

    # CREATE COLOUR MASKS
    
    # blue
    low_blue = np.array([70, 70, 50])
    high_blue = np.array([150, 255, 200])
    blue_mask = cv.inRange(hsv, low_blue, high_blue)
    erode_kernel = np.ones((11,11), np.uint8) 
    blue_mask = cv.erode(blue_mask, erode_kernel, iterations=4)
    dilate_kernel = np.ones((5,5), np.uint8) 
    blue_mask = cv.dilate(blue_mask, dilate_kernel, iterations=1)
    
    # green
    low_grn = np.array([50, 30, 30])
    high_grn = np.array([80, 150, 180])
    grn_mask = cv.inRange(hsv, low_grn, high_grn)
    erode_kernel = np.ones((11,11), np.uint8) 
    grn_mask = cv.erode(grn_mask, erode_kernel, iterations=3)
    # We dilated a lot because there is a lot of green noise
    dilate_kernel = np.ones((5,5), np.uint8) 
    grn_mask = cv.dilate(grn_mask, dilate_kernel, iterations=5)
    
    # yellow
    low_yel = np.array([20, 75, 150]) 
    high_yel = np.array([50, 200, 255])
    yel_mask = cv.inRange(hsv, low_yel, high_yel)
    erode_kernel = np.ones((3,3), np.uint8) 
    yel_mask = cv.erode(yel_mask, erode_kernel, iterations=1)
    dilate_kernel = np.ones((3,3), np.uint8) 
    yel_mask = cv.dilate(yel_mask, dilate_kernel, iterations=1)
    
    # red
    # Red exists at both ends of the color spectrum.
    low_red_one = np.array([0, 150, 50])
    high_red_one = np.array([10, 255, 255])
    red_mask_one = cv.inRange(hsv, low_red_one, high_red_one)

    low_red_two = np.array([150, 150, 50])
    high_red_two = np.array([255, 255, 255])
    red_mask_two = cv.inRange(hsv, low_red_two, high_red_two)

    red_mask = red_mask_one + red_mask_two
    erode_kernel = np.ones((5,5), np.uint8) 
    red_mask = cv.erode(red_mask, erode_kernel, iterations=2)
    dilate_kernel = np.ones((5,5), np.uint8) 
    red_mask = cv.dilate(red_mask, dilate_kernel, iterations=2)
    
    masks = [red_mask, grn_mask, blue_mask, yel_mask]
    
    biggest_area = 0
    biggest_mask = 0
    second_biggest_area = 0
    second_biggest_mask = 0

    for i,mask in enumerate(masks):
        current_area = get_area(mask)
        if current_area > biggest_area:
            second_biggest_area = biggest_area
            second_biggest_mask = biggest_mask
            biggest_area = current_area
            biggest_mask = i+1
        elif current_area > second_biggest_area:
            second_biggest_area = current_area
            second_biggest_mask = i+1

    # 0 = none, 1 = red, 2 = green, 3 = blue, 4 = yellow
    if biggest_area < second_biggest_area*1.2: # arbitary threshold to see if two targets are visible.
        return biggest_mask, second_biggest_mask
    else: 
        return biggest_mask

In [199]:
"""
img0.jpg = yellow target
img4.jpg = red target
img8.jpg = green
img10.jpg = blue
"""

img = cv.imread('C:/Users/Sara/Desktop/robot/new/img6.jpg')
return_orientation(img)

# To see. When we run this from the robot, we may need to convert the robot images to openCV format (BGR)
# IN the function. But It's hard to test without having the robot. So if we start seeing weird results, test that first.

3

## This code is what is contained in the function return_orientation above. Keep for debugging.

In [186]:
# blur images
blur = cv.blur(img,(5,5))
blur0 = cv.medianBlur(blur,5)
blur1= cv.GaussianBlur(blur0,(5,5),0)
blur2= cv.bilateralFilter(blur1,9,75,75)

In [187]:
# cv.imshow('blur2', blur2)
# cv.waitKey(0)

In [188]:
# convert to HSV
hsv = cv.cvtColor(blur1, cv.COLOR_BGR2HSV)

## Create the colour masks

In [189]:
low_blue = np.array([70, 70, 50])
high_blue = np.array([150, 255, 200])
blue_mask = cv.inRange(hsv, low_blue, high_blue)
erode_kernel = np.ones((10,10), np.uint8) 
blue_mask = cv.erode(blue_mask, erode_kernel, iterations=4)
dilate_kernel = np.ones((5,5), np.uint8) 
blue_mask = cv.dilate(blue_mask, dilate_kernel, iterations=1)

In [190]:
low_grn = np.array([50, 30, 30])
high_grn = np.array([80, 150, 180])
grn_mask = cv.inRange(hsv, low_grn, high_grn)
kernel = np.ones((11,11), np.uint8) 
grn_mask = cv.erode(grn_mask, kernel, iterations=3)
# We dilated a lot because there is a lot of green noise
dilate_kernel = np.ones((5,5), np.uint8) 
grn_mask = cv.dilate(grn_mask, dilate_kernel, iterations=5)

In [191]:
low_yel = np.array([20, 75, 150]) 
high_yel = np.array([50, 200, 255])
yel_mask = cv.inRange(hsv, low_yel, high_yel)
erode_kernel = np.ones((3,3), np.uint8) 
yel_mask = cv.erode(yel_mask, erode_kernel, iterations=1)
dilate_kernel = np.ones((3,3), np.uint8) 
yel_mask = cv.dilate(yel_mask, dilate_kernel, iterations=1)

In [192]:
# Red exists at both ends of the color spectrum.
low_red_one = np.array([0, 150, 50])
high_red_one = np.array([10, 255, 255])
red_mask_one = cv.inRange(hsv, low_red_one, high_red_one)

low_red_two = np.array([150, 150, 50])
high_red_two = np.array([255, 255, 255])
red_mask_two = cv.inRange(hsv, low_red_two, high_red_two)

red_mask = red_mask_one + red_mask_two
erode_kernel = np.ones((5,5), np.uint8) 
red_mask = cv.erode(red_mask, erode_kernel, iterations=2)
dilate_kernel = np.ones((5,5), np.uint8) 
red_mask = cv.dilate(red_mask, dilate_kernel, iterations=2)

In [193]:
# # VISUALIZATION OF MASKS
# res = cv.bitwise_and(img,img, mask= blue_mask)
# cv.imshow('blur2', res)
# cv.waitKey(0)

In [194]:
masks = [red_mask, grn_mask, blue_mask, yel_mask]

In [195]:
biggest_area = 0
biggest_mask = 0
second_biggest_area = 0
second_biggest_mask = 0
    
for i,mask in enumerate(masks):
    current_area = get_area(mask)
    if current_area > biggest_area:
        second_biggest_area = biggest_area
        second_biggest_mask = biggest_mask
        biggest_area = current_area
        biggest_mask = i+1
    elif current_area > second_biggest_area:
        second_biggest_area = current_area
        second_biggest_mask = i+1
        
# 0 = none, 1 = red, 2 = green, 3 = blue, 4 = yellow
if biggest_area < second_biggest_area*1.2: # arbitary
    print(biggest_mask, second_biggest_mask)
else: 
    print(biggest_mask)

3
